<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Missing-Data" data-toc-modified-id="Missing-Data-0.0.2"><span class="toc-item-num">0.0.2&nbsp;&nbsp;</span>Missing Data</a></span></li></ul></li></ul></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#Split-Data" data-toc-modified-id="Split-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Split Data</a></span></li><li><span><a href="#Check-Missing-Data" data-toc-modified-id="Check-Missing-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Check Missing Data</a></span></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><span><a href="#Building-Pipeline" data-toc-modified-id="Building-Pipeline-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Building Pipeline</a></span></li></ul></li><li><span><a href="#3.-Prediction" data-toc-modified-id="3.-Prediction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>3. Prediction</a></span></li></ul></div>

In [2]:
import pandas as pd
import numpy as np
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from feature_engine.imputation import CategoricalImputer

In [5]:
SEED=2021

### Load Data

In [6]:
# load train and test data
train_raw = pd.read_csv("./house-prices-advanced-regression-techniques/train.csv", index_col='Id')
test_raw = pd.read_csv("./house-prices-advanced-regression-techniques/test.csv", index_col='Id')

In [7]:
print("The shape of train: {}".format(train_raw.shape))
print("The shape of test: {}".format(test_raw.shape))

The shape of train: (1460, 80)
The shape of test: (1459, 79)


### Missing Data 

In [8]:
def getColInfo(train, dtype=None, cardinality_threshold=0):
    train_dtypes = [trai[dt].dtype for dt in train.columns]
    nuniques = [train[col].nunique() for col in train.columns]
    res = pd.DataFrame({'Columns Names': train.columns,
                                 'Dtypes': train_dtypes,
                                 'Unique Value Numbers': nuniques })
    if dtype != None:
        res = res.loc[res['Dtypes']==dtype]
    res = pd.DataFrame(res.loc[res['Unique Value Numbers'] >= cardinality_threshold])
    return(res.sort_values(by=['Unique Value Numbers'], axis=0))

def getMissingData(train):
    all_data_na = (train.isnull().sum() / len(train)) * 100
    all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
    all_data_na_dtypes = [train[dt].dtype for dt in all_data_na.index]
    nuniques = train[all_data_na.index].nunique()
    missing_data = pd.DataFrame({'Dtypes': all_data_na_dtypes,
                                 'Missing Ratio' : all_data_na,
                                 'Unique Value Numbers': nuniques })
    if missing_data.shape[0] == 0:
        return("no missing vlaue")
    return(missing_data)

In [9]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

# Preprocessing

## Split Data

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(train_raw.drop('SalePrice', axis=1), 
                                                      train_raw['SalePrice'], 
                                                      test_size=0.1, 
                                                      random_state=99)

## Check Missing Data

In [11]:
getColInfo(X_train)

,Columns Names,Dtypes,Unique Value Numbers
8,Utilities,object,1
40,CentralAir,object,2
4,Street,object,2
5,Alley,object,2
59,GarageFinish,object,3
...,...,...,...
37,TotalBsmtSF,int64,679
42,1stFlrSF,int64,710
36,BsmtUnfSF,int64,732
45,GrLivArea,int64,811


In [12]:
missing_data = getMissingData(X_train);missing_data

,Dtypes,Missing Ratio,Unique Value Numbers
PoolQC,object,99.467275,3
MiscFeature,object,96.194825,4
Alley,object,93.607306,2
Fence,object,80.365297,4
FireplaceQu,object,47.792998,5
LotFrontage,float64,17.351598,109
GarageType,object,5.859970,6
GarageYrBlt,float64,5.859970,97
GarageFinish,object,5.859970,3
GarageQual,object,5.859970,5


In [13]:
# columns need to be dropped
# columns that missing ratio greater than threshold are dropped (%)
col_drop_threshold = 5

cols_dropped = missing_data.index[missing_data['Missing Ratio'] > col_drop_threshold]
cols_dropped = list(cols_dropped)
cols_dropped.append('Utilities')

Notice that column "Utilities" only have 1 kind of value and it does not have missing value thus it is dropped.

In [17]:
# columns survived
cols_survived = X_train.columns.drop(cols_dropped)
cols_survived

Index(['MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageCars', 'GarageArea',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
      

In [18]:
# split categorical and numerical variables from cols_survived
X_train_survived = X_train.copy()[cols_survived]

# categorical column names
cat_cols_survived = list(X_train_survived.columns[X_train_survived.dtypes == 'object'])

# numerical column names
num_cols_survived = list(set(X_train_survived) - set(cat_cols_survived))

In [19]:
# check all numerical variables(not just columns with missing values in this training set)  
# to determine the imputing values for each columns
X_train[num_cols_survived]

,Fireplaces,BsmtFinSF1,OverallCond,1stFlrSF,BedroomAbvGr,BsmtHalfBath,HalfBath,2ndFlrSF,MSSubClass,KitchenAbvGr,...,WoodDeckSF,FullBath,YearBuilt,MiscVal,3SsnPorch,GarageArea,MasVnrArea,OverallQual,TotRmsAbvGrd,MoSold
Id,,,,,,,,,,,,,,,,,,,,,
147,0,209,7,875,2,0,0,0,30,1,...,48,1,1931,0,0,180,0.0,5,5,11
1237,0,0,5,764,2,0,1,862,160,1,...,0,2,2003,0,0,474,106.0,7,6,6
799,2,0,5,1966,4,0,1,1174,60,1,...,144,3,2008,0,0,820,860.0,9,11,7
254,0,270,7,1302,3,1,0,0,80,1,...,333,2,1964,0,0,309,108.0,6,7,10
274,1,620,6,1632,3,0,0,0,20,1,...,289,2,1958,0,0,338,183.0,6,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,1,423,5,1390,4,0,0,304,50,1,...,342,2,1925,0,0,576,0.0,6,7,6
1193,0,0,8,842,3,0,0,630,50,1,...,0,1,1925,0,0,250,0.0,5,6,7
1210,1,1220,5,1660,3,0,0,0,20,1,...,322,2,2006,0,0,500,420.0,8,8,5


In [20]:
# check all categorical variables
X_train[cat_cols_survived]

,MSZoning,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,...,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
147,RM,Pave,Reg,Lvl,Corner,Gtl,BrkSide,Norm,Norm,1Fam,...,Unf,GasA,TA,Y,FuseA,TA,Typ,Y,WD,Normal
1237,RL,Pave,Reg,Lvl,Inside,Gtl,NridgHt,Norm,Norm,Twnhs,...,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Y,WD,Normal
799,RL,Pave,Reg,Lvl,Inside,Gtl,NridgHt,Norm,Norm,1Fam,...,Unf,GasA,Ex,Y,SBrkr,Ex,Typ,Y,New,Partial
254,RL,Pave,Reg,Lvl,Inside,Gtl,NAmes,Norm,Norm,1Fam,...,ALQ,GasA,Ex,Y,SBrkr,Gd,Min1,Y,CWD,Normal
274,RL,Pave,Reg,Lvl,Inside,Gtl,NAmes,Feedr,Norm,1Fam,...,LwQ,GasA,Gd,Y,SBrkr,TA,Min1,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,RL,Pave,Reg,Bnk,Inside,Gtl,SWISU,Norm,Norm,1Fam,...,Unf,GasA,Fa,Y,SBrkr,TA,Typ,Y,WD,Normal
1193,RM,Pave,Reg,Lvl,Inside,Gtl,OldTown,Norm,Norm,1Fam,...,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,P,WD,Normal
1210,RL,Pave,IR1,Lvl,CulDSac,Gtl,Somerst,RRNn,Norm,1Fam,...,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Y,New,Partial


In [21]:
# num_col_mappers = [([n],[SimpleImputer(missing_values=np.nan, strategy='median')]) for n in num_cols_survived]

In [57]:
num_col_mappers = [(num_cols_survived,[SimpleImputer(missing_values=np.nan, strategy='median')])]

In [23]:
# cat_col_mappers = [([c],[SimpleImputer(missing_values=np.nan, strategy='most_frequent'),
#                          OneHotEncoder(handle_unknown='ignore')]) for c in cat_cols_survived]

In [87]:
cat_col_mappers = [([cat_cols_survived],[CategoricalImputer(imputation_method='frequent'),
                         OneHotEncoder(handle_unknown='ignore')])]

In [89]:
cols_mapper = DataFrameMapper(num_col_mappers + cat_col_mappers)

In [90]:
type(cols_mapper)

sklearn_pandas.dataframe_mapper.DataFrameMapper

# Modeling

In [91]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [28]:
rmsle_cv()

TypeError: rmsle_cv() missing 1 required positional argument: 'model'

## Building Pipeline

In [92]:
my_train_pipeline = Pipeline([
    ('preprecess', cols_mapper)
    #('random_forest', RandomForestRegressor())
])


param_grid = dict(random_forest__n_estimators=[500],
                random_forest__max_depth=[4,5,6,7,8,10,12],
                random_forest__criterion=['squared_error'],
                random_forest__min_samples_leaf=[5,10,20],
                random_forest__oob_score=[True],
                random_forest__random_state=[SEED])

In [30]:
#preproess_pipe = make_pipeline(cols_mapper, RandomForestRegressor())

In [68]:
type(y_train_trans[1])

numpy.float64

In [93]:
my_train_pipeline.fit(X_train_survived)

KeyError: "[['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']]: None of [Index([('MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition')], dtype='object')] are in the [columns]"

In [94]:
grid = GridSearchCV(my_train_pipeline, scoring='neg_mean_squared_error', cv=5, param_grid=param_grid)

# transform y
y_train_trans = np.log1p(y_train)
grid.fit(X_train_survived.values,y_train_trans.values)

ValueError: Invalid parameter random_forest for estimator Pipeline(steps=[('preprecess',
                 DataFrameMapper(drop_cols=[],
                                 features=[(['Fireplaces', 'BsmtFinSF1',
                                             'OverallCond', '1stFlrSF',
                                             'BedroomAbvGr', 'BsmtHalfBath',
                                             'HalfBath', '2ndFlrSF',
                                             'MSSubClass', 'KitchenAbvGr',
                                             'EnclosedPorch', 'GrLivArea',
                                             'OpenPorchSF', 'LotArea',
                                             'TotalBsmtSF', 'LowQualFinSF',
                                             'YrSold', 'GarageCars',
                                             'ScreenPorch', 'PoolArea',
                                             'BsmtFinSF2', 'BsmtFu...
                                              'BldgType', 'HouseS

In [ ]:
y_train_trans.values

# 3. Prediction

In [29]:
df=pd.DataFrame({"col1":['d1','dfd',np.nan,'tr'], 
                 "col2":['dfs',np.nan,'po','po'],
                 "col3":[12,13,10,np.nan]});df

,col1,col2,col3
0,d1,dfs,12.0
1,dfd,NaN,13.0
2,NaN,po,10.0
3,tr,po,NaN


In [25]:
cat_im = CategoricalImputer(imputation_method='frequent')

In [26]:
cat_im.fit_transform(df)

ValueError: Variable col1 contains multiple frequent categories.

In [27]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)


In [49]:
im = DataFrameImputer()
new_df = im.fit_transform(df)

In [71]:
df

,col1,col2,col3
0,d1,dfs,12.0
1,dfd,NaN,13.0
2,NaN,po,10.0
3,tr,po,NaN


In [68]:
pd.get_dummies(df,prefix='col1')

,col3,col1_d1,col1_dfd,col1_tr,col1_dfs,col1_po
0,12.0,1,0,0,1,0
1,13.0,0,1,0,0,0
2,10.0,0,0,0,0,1
3,NaN,0,0,1,0,1


In [89]:
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(df)
df.encoded = pd.DataFrame(ohe.transform(df).toarray(), columns=ohe.get_feature_names_out(), index=)


NameError: name 'X_train' is not defined

In [76]:
ohe.get_feature_names_out()

array(['col1_d1', 'col1_dfd', 'col1_tr', 'col1_nan', 'col2_dfs',
       'col2_po', 'col2_nan', 'col3_10.0', 'col3_12.0', 'col3_13.0',
       'col3_nan'], dtype=object)

In [95]:
df.drop(['col1'],axis=1)

,col2,col3
0,dfs,12.0
1,NaN,13.0
2,po,10.0
3,po,NaN


In [92]:
pd.concat([df,df], axis=1)

,col1,col2,col3,col1,col2,col3
0,d1,dfs,12.0,d1,dfs,12.0
1,dfd,NaN,13.0,dfd,NaN,13.0
2,NaN,po,10.0,NaN,po,10.0
3,tr,po,NaN,tr,po,NaN
